In [62]:
import random
import torch
import pickle

import re

import pandas
pandas.set_option('display.max_rows', None)

from sentence_transformers import SentenceTransformer, models
from bertopic import BERTopic

## Data importing and cleaning

* Remove links, images, @usernames and RTs indicating re-tweeting.
* Remove all content which is empyt after this

In [63]:
data = pandas.read_csv("realDonaldTrump_in_office.csv")

## remove links and images
data['Tweet.Text'] = data['Tweet.Text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
data['Tweet.Text'] = data['Tweet.Text'].replace(r'pic.twitter.com\S+', '', regex=True)

## remove usernames
data['Tweet.Text'] = data['Tweet.Text'].replace(r'@\S+', '', regex=True)

## remove RTs
data['Tweet.Text'] = data['Tweet.Text'].replace(r'RT', '', regex=True)

data['Tweet.Text'] = data['Tweet.Text'].str.strip()

## remove items which have no remaining content after cleaning it
data = data.drop( data[ data['Tweet.Text'] == '' ].index )

data.head()

,Unnamed: 0,ID,Time,Tweet.URL,Tweet.Text
0,1,@realDonaldTrump,2017-01-20 06:31,https://twitter.com/realDonaldTrump/status/82...,It all begins today! I will see you at 11:00 A...
1,2,@realDonaldTrump,2017-01-20 11:51,https://twitter.com/realDonaldTrump/status/82...,Today we are not merely transferring power fro...
2,3,@realDonaldTrump,2017-01-20 11:51,https://twitter.com/realDonaldTrump/status/82...,"power from Washington, D.C. and giving it back..."
3,4,@realDonaldTrump,2017-01-20 11:52,https://twitter.com/realDonaldTrump/status/82...,What truly matters is not which party controls...
4,5,@realDonaldTrump,2017-01-20 11:53,https://twitter.com/realDonaldTrump/status/82...,"January 20th 2017, will be remembered as the d..."


## BERTopics created

* Resetting seeds for each model, extracting the word embeddings and then running the analysis with as close to default settings as possible

In [64]:
def create_model( model_name ):

    torch.manual_seed(0)
    random.seed(0)

    word_embedding_model = models.Transformer(model_name, max_seq_length=256)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

    transformer_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    topic_model = BERTopic(embedding_model=transformer_model )
    topic_model.fit_transform(data['Tweet.Text'])

    reduced_topics = topic_model.reduce_topics(data['Tweet.Text'], nr_topics="auto")
    
    pickle.dump( ( reduced_topics, topic_model, transformer_model ) , file = open( f"{model_name.replace('/', '_').replace('..', '')}", "wb") )
    return reduced_topics, topic_model, transformer_model

In [65]:
capitalist, capitalist_full, capitalist_transformer = create_model("microsoft/deberta-v3-base")
marxist, marxist_full, marxist_transformer = create_model("../models/microsoft_deberta-v3-base-finetuned-masked-model")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
TOKENIZERS_PARALLELISMTo disable this warning, you can either:
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable 	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISMhuggingface/tokenizers: The current process just got forked, after parallelism has already been us

## Basic summaries

In [67]:
capitalist.merge_topics(data['Tweet.Text'], [-1, 0])
capitalist.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,19428,-1_the_to_and_of,"[the, to, and, of, in, is, for, that, on, are]",[It is amazing that I became President of the ...
1,0,205,0_great_true_wow_thanks,"[great, true, wow, thanks, job, book, amazing,...","[Great!, Great!, Great!]"
2,1,127,1_thank_you_great_nice,"[thank, you, great, nice, honor, job, keep, wo...","[THANK YOU GREAT JOB!!, Great news, thank you..."
3,2,97,2_live_delivers_remarks_potus,"[live, delivers, remarks, potus, president, sa...","[LIVE: President delivers remarks on testing,..."
4,3,85,3_again_make_america_great,"[again, make, america, great, watcher, volunte...","[MAKE AMERICA GREAT AGAIN!, MAKE AMERICA GREAT..."
5,4,84,4_thank_you_000_matt,"[thank, you, 000, matt, payrolls, adp, 219, es...","[Thank you Mark!, Thank you John!, THANK YOU B..."
6,5,78,5_interviewed_00_enjoy_tonight,"[interviewed, 00, enjoy, tonight, at, eastern,...",[I will be interviewed by tonight at 9:00 P.M...
7,6,77,6_great_usa_job_agree,"[great, usa, job, agree, wow, dirty, congratul...","[WOW, they got caught. End the Witch Hunt now!..."
8,7,77,7_true_watch_enjoy_now,"[true, watch, enjoy, now, address, on, weekly,...","[Thanks - TRUE!, Watch on NOW. Enjoy!, Than..."
9,8,75,8_pleased_announce_secretary_united,"[pleased, announce, secretary, united, of, am,...",[We are pleased to announce that Matthew G. Wh...


In [69]:
marxist.merge_topics(data['Tweet.Text'], [-1, 0])
marxist.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,19006,-1_the_to_and_of,"[the, to, and, of, in, is, for, that, on, are]","[....and other really bad people, SPIED ON MY ..."
1,0,462,0_via_trump_biden_election,"[via, trump, biden, election, fbi, to, clinton...",[EXCLUSIVE: Report Shows FBI Official Received...
2,1,118,1_thank_you_matt_franklin,"[thank, you, matt, franklin, rudy, brad, steve...","[Thank you!, Thank you!, Thank you!]"
3,2,101,2_true_wow_thanks_great,"[true, wow, thanks, great, amazing, yes, inter...","[True!, Thanks - TRUE!, Thanks - TRUE!]"
4,3,99,3_interviewed_00_enjoy_tonight,"[interviewed, 00, enjoy, tonight, at, be, east...",[Will be interviewed by tonight at 9:00 P.M. ...
5,4,89,4_see_soon_way_state,"[see, soon, way, state, landed, everyone, head...","[On my way, see you soon!, On my way to the Gr..."
6,5,87,5_thank_you_love_nice,"[thank, you, love, nice, honor, so, true, grea...","[Thank you. I love you too!, Great news, thank..."
7,6,83,6_thank_maga_you_maga2020,"[thank, maga, you, maga2020, kag2020, missoula...","[THANK YOU! #MAGA, THANK YOU! #MAGA, THANK YOU..."
8,7,82,7_again_make_america_great,"[again, make, america, great, keep, making, pr...","[MAKE AMERICA GREAT AGAIN!, MAKE AMERICA GREAT..."
9,8,79,8_thank_you_mike_great,"[thank, you, mike, great, dirty, usa, group, k...","[Thank you! ‚Ä¶, Thank you! ‚Ä¶, Thank you! ‚Ä¶]"


In [70]:
capitalist.visualize_barchart().write_image("capitalist-barchart.pdf")
capitalist.visualize_barchart()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.07565869594656666, 0.11574989500462039, 0.1416706408908798,
                    0.17545301089516321, 0.21543192610683157],
              'xaxis': 'x',
              'y': [job  , thanks  , wow  , true  , great  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0477240540442246, 0.052143408338170084, 0.08976330254451315,
                    0.15696593854790739, 0.21106966988582274],
              'xaxis': 'x2',
              'y': [honor  , nice  , great  , you  , thank  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.032456916427228, 0.03431254481067215, 0.059651621593134634,
                    0.07356566969230031, 0.10141705640475437],
              'xaxis': 'x3',
              'y': [president  , potus  , remarks  , delivers  , live  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.15242395098175554, 0.21802043031985854, 0.3738794452492452,
                    0.42454479982594306, 0.47069006809264363],
              'xaxis': 'x4',
              'y': [watcher  , great  , america  , make  , again  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03934366533414377, 0.0476967550235159, 0.06284248072922625,
                    0.183242110181173, 0.2889585443870146],
              'xaxis': 'x5',
              'y': [payrolls  , matt  , 000  , you  , thank  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.1447823826876693, 0.1633193533359888, 0.26781404441312173,
                    0.3056430602917839, 0.3163532471880701],
              'xaxis': 'x6',
              'y': [at  , tonight  , enjoy  , 00  , interviewed  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04277717403523319, 0.04767409635866026, 0.04845908211453649,
                    0.0498502156857567, 0.08738939702267479],
              'xaxis': 'x7',
              'y': [wow  , agree  , job  , usa  , great  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0441237270031217, 0.04471555314457454, 0.050310681929190484,
                    0.057233074697706124, 0.07331188067570685],
              'xaxis': 'x8',
              'y': [address  , now  , enjoy  , watch  , true  ],
              'yaxis': 'y8'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
 

In [71]:
marxist.visualize_barchart().write_image("marxist-barchart.pdf")
marxist.visualize_barchart()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.011486646669622172, 0.012431288168237607,
                    0.014032090470058071, 0.021595620646452606,
                    0.03214779482457904],
              'xaxis': 'x',
              'y': [fbi  , election  , biden  , trump  , via  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03822231902499201, 0.040056509434912874,
                    0.06357316045692528, 0.4032863047961809, 0.6562604331023801],
              'xaxis': 'x2',
              'y': [rudy  , franklin  , matt  , you  , thank  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.13946770495253558, 0.17732415982026678, 0.29732439500277547,
                    0.3490814714190747, 0.5487543685666976],
              'xaxis': 'x3',
              'y': [amazing  , great  , thanks  , wow  , true  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.1503196890088654, 0.20902465013474367, 0.2642754522822194,
                    0.26441744667195083, 0.3654835343169592],
              'xaxis': 'x4',
              'y': [at  , tonight  , enjoy  , 00  , interviewed  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.06464441677646196, 0.06526613725451473, 0.06801523916691093,
                    0.1327083304167988, 0.13293888868930223],
              'xaxis': 'x5',
              'y': [landed  , state  , way  , soon  , see  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.06530456897197415, 0.08793295144748492, 0.11454182402137815,
                    0.2098791374414593, 0.2561493633649986],
              'xaxis': 'x6',
              'y': [honor  , nice  , love  , you  , thank  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0447877625689645, 0.06183131995664561, 0.21232735643600442,
                    0.3027329830743231, 0.3152837391916796],
              'xaxis': 'x7',
              'y': [kag2020  , maga2020  , you  , maga  , thank  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.07728174779490199, 0.2841029495572386, 0.5026538830908789,
                    0.5118266266662633, 0.5756723631256953],
              'xaxis': 'x8',
              'y': [keep  , great  , america  , make  , again  ],
              'yaxis': 'y8'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font':

In [72]:
marxist.get_representative_docs(3)

['Will be interviewed by  tonight at 9:00 P.M. on  Enjoy!!!',
 'I will be interviewed by  tonight at 9:00 P.M. on  Enjoy!',
 'Will be interviewed by  on  tonight at 8:00 P.M. Enjoy!']

## Focus on the Nasdaq topic as an example

In [100]:
marxist.get_representative_docs(55)

['DOW, S&amp;P 500 and NASDAQ close at record highs! #MAGA',
 '“DOW, NASDAQ, S&amp;P 500 CLOSE AT RECORD HIGHS”',
 '“DOW, NASDAQ, S&amp;P 500 CLOSE AT RECORD HIGHS”']

In [109]:
topics_over_time = marxist.topics_over_time( data['Tweet.Text'], data['Time'], nr_bins = 50  )
marxist.visualize_topics_over_time(topics_over_time, topics = [55 ])

2024-06-25 13:43:42,889 - BERTopic - WARNING: There are more than 100 unique timestamps (i.e., 18613) which significantly slows down the application. Consider setting `nr_bins` to a value lower than 100 to speed up calculation. 


In [107]:
for topic in capitalist.find_topics('NASDAQ')[0]:
    print( topic )
    print( capitalist.get_representative_docs( topic ) )

32
['Congratulations', 'Congratulations', 'Congratulations']
2
['So true!', 'So true!', 'So true']
49
['OBAMAGATE!', 'OBAMAGATE!', 'OBAMAGATE!']
43
['Presidential Harassment!', 'Presidential Harassment!', 'Presidential Harassment!']
20
['READ THE TRANSCRIPTS!', 'Read the transcripts!', 'Read the Transcripts!']
